In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# ChromeDriver 경로 설정 (macOS 사용자의 경우 절대 경로로 설정 필요)
chromedriver_path = '/opt/homebrew/bin/chromedriver'  # 실제 chromedriver 경로로 변경

# Selenium 4.x에서 Service 객체 사용
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

# 네이버 뉴스 페이지
link = 'https://news.naver.com/breakingnews/section/101/259?date='
date = '20241202'
main_link = link + date

# 크롬드라이버 실행
driver.get(main_link)
time.sleep(3)

# 기사 더보기 버튼 클릭
try:
    more_button = driver.find_element(By.CLASS_NAME, 'section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON')
    while True:
        try:
            more_button.click()
            time.sleep(3)
        except:
            break
except Exception as e:
    print("More button not found or no more articles to load:", e)

# 기사 정보 추출
articles = driver.find_elements(By.CLASS_NAME, 'sa_text_title')
data = []
for i, article in enumerate(articles):
    title = article.text.strip()
    link = article.get_attribute('href')
    data.append([i + 1, title, link])

# DataFrame 생성
Main_link = pd.DataFrame(data, columns=['number', 'title', 'link'])

# 엑셀 파일 저장
excel_name = f'news_{date}.xlsx'
Main_link.to_excel(excel_name, index=False)
print(f'엑셀 파일 저장 완료: {excel_name}')

# 브라우저 종료
driver.quit()


엑셀 파일 저장 완료: news_20241202.xlsx


In [15]:
Main_link

,number,title,link
0,1,"동양생명, '(무)수호천사 미니 독감케어보험' 출시",https://n.news.naver.com/mnews/article/014/000...
1,2,"KB금융, 테크포럼 개최…최신 디지털금융 기술 공유",https://n.news.naver.com/mnews/article/421/000...
2,3,"주주 충실 의무, 자본시장법에 담는다…절차 규제 집중(종합)",https://n.news.naver.com/mnews/article/003/001...
3,4,"행정공제회, 회원 복지 가족캠프 성료",https://n.news.naver.com/mnews/article/018/000...
4,5,[속보] 합병·분할시 소액주주보호…자본시장법 정부안 이번주 국회제출,https://n.news.naver.com/mnews/article/009/000...
...,...,...,...
281,282,[주간경제전망] 3분기 GDP 확정치 나온다… 코스피 향방은?,https://n.news.naver.com/mnews/article/417/000...
282,283,[오늘의 주요일정]금융·증권(12월2일 월요일),https://n.news.naver.com/mnews/article/003/001...
283,284,[단독]“계파 상관없이 능력인사… 조직문화 과감하게 혁신”,https://n.news.naver.com/mnews/article/020/000...
284,285,"은행 대출금리 잇단 인하… 국민銀, 최대 0.19%P 내려",https://n.news.naver.com/mnews/article/020/000...


In [22]:
from bs4 import BeautifulSoup
import requests
from openpyxl import *
import urllib

# 첫 번째 코드에서 지정한 뉴스의 링크들이 담긴 파일
link = pd.read_excel('news_20241202.xlsx')
# 엑셀 파일의 빈칸이 있기에 최종 결과파일을 별도로 생성
excel_name = 'news_detail_20241202.xlsx'
Main_link = list(link['link'])

# number: 기사 순서, title: 기사 제목, information: 본문 내용, link: 기사의 링크
Information = pd.DataFrame({'number': [], 'title': [], 'information': [], 'link': []})
# 본문 내용을 추가하기 전이기 때문에 미리 나머지 내용을 담아둠
Information['number'] = link['number']
Information['title'] = link['title']
Information['link'] = link['link']
information = []

for main_link in Main_link:
    # 기사가 전체적으로 2개의 구조를 가지고 있음 (게임/리뷰 카테고리에 한하여)
    # 하나의 구조를 기준으로 삼고, 해당 부분에서 오류가 발생하면 다음 구조의 기사로 판단
    try:
        response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            # 기사의 본문 내용만 담고 있는 부분
            info = soup.find('div', {'id': 'newstct_article'}).text.strip()
            # 기사 내용 데이터 분석을 위해 줄바꿈을 띄어쓰기로 변경
            info = info.replace('\n', ' ')
            information.append(info)
    except:
        # 다른 구조에서 재 크롤링 코드
        # 여기서 오류가 나는 경우는 게임/리뷰 기사가 아닌 다른 카테고리의 기사로 판단
        try:
            response = requests.get(main_link, headers={"User-Agent": "Mozilla/5.0"})
            if response.status_code == 200:
                html = response.content
                soup = BeautifulSoup(html, 'html.parser')
                # 기사의 본문 내용을 담고 있는 부분
                info = soup.find('div', {'id': 'newsEndContents'}).text.strip()
                info = info.replace('\n', ' ')
                # 해당 구조의 기사는 기자의 정보가 본문과 무조건 같이 존재
                # 기자의 정보 부분은 필요가 없기 때문에 기자 정보의 기준점이 되는 부분을 찾음
                # 기자의 정보 기준이 기재정보라는 단어이기 때문에 그 이후는 삭제
                '''
                end = info.index('기재정보')
                info = info[:end]
                '''
                information.append(info)
        except Exception as e:
            info = ''
            information.append(info)
            # 오류가 발생하는 이유와 발생하는 링크를 출력하여 오류를 확인하는 장치
            # print(e)
            # print(main_link)

Information['information'] = information

with pd.ExcelWriter(excel_name) as writer:
    Information.to_excel(writer, sheet_name='결과값', index=False)

In [23]:
Information

,number,title,information,link
0,1,"동양생명, '(무)수호천사 미니 독감케어보험' 출시",,https://n.news.naver.com/mnews/article/014/000...
1,2,"KB금융, 테크포럼 개최…최신 디지털금융 기술 공유",,https://n.news.naver.com/mnews/article/421/000...
2,3,"주주 충실 의무, 자본시장법에 담는다…절차 규제 집중(종합)",,https://n.news.naver.com/mnews/article/003/001...
3,4,"행정공제회, 회원 복지 가족캠프 성료",,https://n.news.naver.com/mnews/article/018/000...
4,5,[속보] 합병·분할시 소액주주보호…자본시장법 정부안 이번주 국회제출,,https://n.news.naver.com/mnews/article/009/000...
...,...,...,...,...
281,282,[주간경제전망] 3분기 GDP 확정치 나온다… 코스피 향방은?,,https://n.news.naver.com/mnews/article/417/000...
282,283,[오늘의 주요일정]금융·증권(12월2일 월요일),,https://n.news.naver.com/mnews/article/003/001...
283,284,[단독]“계파 상관없이 능력인사… 조직문화 과감하게 혁신”,,https://n.news.naver.com/mnews/article/020/000...
284,285,"은행 대출금리 잇단 인하… 국민銀, 최대 0.19%P 내려",,https://n.news.naver.com/mnews/article/020/000...


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
from openpyxl import *

# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 표시하지 않음
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 엑셀 파일 읽기
link = pd.read_excel('news_20241202.xlsx')
excel_name = 'news_detail_20241202.xlsx'
Main_link = list(link['link'])

# 데이터 프레임 생성
Information = pd.DataFrame({'number': [], 'title': [], 'information': [], 'link': []})
Information['number'] = link['number']
Information['title'] = link['title']
Information['link'] = link['link']
information = []

for main_link in Main_link:
    try:
        driver.get(main_link)
        # 첫 번째 구조로 기사 본문 크롤링
        try:
            info = driver.find_element(By.ID, 'newstct_article').text.strip()
            info = info.replace('\n', ' ')
            information.append(info)
        except:
            # 두 번째 구조로 기사 본문 크롤링
            try:
                info = driver.find_element(By.ID, 'newsEndContents').text.strip()
                info = info.replace('\n', ' ')
                if '기재정보' in info:
                    end = info.index('기재정보')
                    info = info[:end]
                information.append(info)
            except Exception as e:
                # 오류가 발생한 경우 빈 문자열 추가
                information.append('')
                print(f"오류 발생: {e} 링크: {main_link}")
    except Exception as e:
        # 전체적으로 오류 발생 시 빈 문자열 추가
        information.append('')
        print(f"전체 오류 발생: {e} 링크: {main_link}")

# 데이터 프레임에 본문 추가
Information['information'] = information

# 엑셀 파일로 저장
with pd.ExcelWriter(excel_name) as writer:
    Information.to_excel(writer, sheet_name='결과값', index=False)

# 드라이버 종료
driver.quit()


오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="newsEndContents"]"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d47648 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x0000000104d3fea8 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x00000001047ac104 cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001047ee364 cxxbridge1$string$len + 359360
4   chromedriver                        0x0000000104827bd0 cxxbridge1$string$len + 594988
5   chromedriver                        0x00000001047e2f54 cxxbridge1$string$len + 313264
6   chromedriver                        0x00000001047e3ba4 cxxbridge1$string$len + 316416
7   chromedriver                        0x0000000104d121e

In [21]:
Information

,number,title,information,link
0,1,"동양생명, '(무)수호천사 미니 독감케어보험' 출시",,https://n.news.naver.com/mnews/article/014/000...
1,2,"KB금융, 테크포럼 개최…최신 디지털금융 기술 공유",,https://n.news.naver.com/mnews/article/421/000...
2,3,"주주 충실 의무, 자본시장법에 담는다…절차 규제 집중(종합)",,https://n.news.naver.com/mnews/article/003/001...
3,4,"행정공제회, 회원 복지 가족캠프 성료",,https://n.news.naver.com/mnews/article/018/000...
4,5,[속보] 합병·분할시 소액주주보호…자본시장법 정부안 이번주 국회제출,,https://n.news.naver.com/mnews/article/009/000...
...,...,...,...,...
281,282,[주간경제전망] 3분기 GDP 확정치 나온다… 코스피 향방은?,,https://n.news.naver.com/mnews/article/417/000...
282,283,[오늘의 주요일정]금융·증권(12월2일 월요일),,https://n.news.naver.com/mnews/article/003/001...
283,284,[단독]“계파 상관없이 능력인사… 조직문화 과감하게 혁신”,,https://n.news.naver.com/mnews/article/020/000...
284,285,"은행 대출금리 잇단 인하… 국민銀, 최대 0.19%P 내려",,https://n.news.naver.com/mnews/article/020/000...
